In [ ]:
# IMPORT YOUR LIBRARIES HERE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from math import comb

# Sklearn import
from sklearn.model_selection import train_test_split # Splitting the data set
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler # Normalization and standard scaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # Encoders
from sklearn.preprocessing import PolynomialFeatures # Polynomial features
from sklearn.preprocessing import FunctionTransformer # Transform data
from sklearn.linear_model import LinearRegression # Regression linear model
from sklearn.linear_model import Lasso, Ridge, ElasticNet  # Regularization
from sklearn.linear_model import LassoCV, RidgeCV  # Regularization and Cross-Validation
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV # Logistic regression model
from sklearn.neighbors import KNeighborsClassifier # KNN Algorithm
from sklearn.tree import DecisionTreeClassifier, plot_tree # Decision Trees
from sklearn.model_selection import GridSearchCV   # Grid search for cross validation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error # Metrics regression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score  # Metrics classifier

# Assignment 

Welcome to the assignment! 

You will have to implement regression and classification algorithms, applying these methods to the topics of agriculture, food, water, and health. More precisely, you will try to:
- predict crop yields using data on weather and fertilizer use;
- predict the potability of water using data on the physical and chemical properties of water.




Once you are done you have to submit your notebook here: 

There is no quiz, so your grade will depend only on your notebook.

If there is need for further clarifications on the questions, after the assignment is released, we will update this file, so make sure you check the git repository for updates.

Good luck!

## Linear regression: predicting crop yields

Agriculture plays a critical role in the global economy. Given the ongoing growth of the world population, it is imperative to comprehend crop yield at a global level in order to tackle food security issues and mitigate the effects of climate change.

Crop yield prediction is an important agricultural problem. The Agricultural yield primarily depends on weather conditions (rain, temperature, etc) and fertilizers use. Having precise information regarding the historical crop yield is critical for making informed decisions regarding agricultural risk management and future projections.

Some E4S publications on the topic of food:
- [Threats to Nitrogen Fertilizer, Opportunities to Cultivate Sustainable Practices?](https://e4s.center/resources/reports/threats-to-nitrogen-fertilizer-opportunities-to-cultivate-sustainable-practices/)
- [True cost of food as a lever to transform the Swiss food system](https://e4s.center/resources/reports/true-cost-of-food-as-a-lever-to-transform-the-swiss-food-system/)

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Sustainable_Development_Goal_02ZeroHunger.svg/800px-Sustainable_Development_Goal_02ZeroHunger.svg.png' width="200">

We will use data obtained from the [FAO](http://www.fao.org/home/en/) (Food and Agriculture Organization) and [World Data Bank](https://data.worldbank.org/), and gathered in the [Crop Yield Prediction Dataset](https://www.kaggle.com/datasets/patelris/crop-yield-prediction-dataset).

Our goal is to predict the crop yields using the temperature, rain fall, country, and type of crops.


### Question 1: Load and Discover the dataset

- Load the data in a dataframe. The url link is provided below. Display the first 10 observations and the types of data **1 point**

In [ ]:
url_yield = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/yield_df.csv'

# Import data in dataframe
food = pd.read_csv(url_yield)
# Display first 10 observations
display(food.head(10))
# Print the data types
print(food.dtypes)

We have some numerical variables (pesticides, rainfall, temperature, yield, and year) and some categorical variables (area and item).

- Print the list of countries ('Area') and years available in the dataset **1 point**

In [ ]:
# List of unique countries
country_list = food.Area.unique() 

# Print result
## We use join to print our lists element by element, separated by a comma:
print('Our dataset contains {} countries: '.format(len(country_list)) + ', '.join(country_list))

In [ ]:
# List of unique years as strings
## We use a list comprehension to convert our years into string:
years_str = [str(y) for y in food.Year.unique()]

# Print result
print('Our dataset contains the following years: '
      +', '.join(years_str))

- Print the list of 'Item' in the dataset. You should obtain a list of 10 crops, which are among the most consumed in the world **1 point**

In [ ]:
# List of unique item
item_list = food.Item.unique() 

# Print result
print('Our dataset contains {} types of crops: '.format(len(item_list)) 
      + ', '.join(item_list))

- Display summary statistics for the columns: 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp'. How many observations do we have? **1 point**

*Hint:* You can extract the columns 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp' in a new dataframe since we will reuse it in the following questions

In [ ]:
# Extract columns of interest
food_data = food.loc[:,['hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp']]

# Summary statistics
display(food_data.describe())

# Number of observations
print('Our dataset contains {} observations.'.format(food.shape[0]))

- Display a heatmap of the correlation matrix between the columns: 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp' **1 point**

In [ ]:
# We rename the columns of our dataframe for clarity
food_data.rename(columns = {'hg/ha_yield':'Yield',
                           'average_rain_fall_mm_per_year':'Rainfall',
                           'pesticides_tonnes':'Pesticides',
                           'avg_temp':'Temperature'},
                 inplace=True)

# Correlation matrix
food_corr = food_data.corr()

# Heatmap
plt.figure(figsize=(5, 4))
plt.title("Correlation matrix")
sns.heatmap(food_corr.round(decimals=3), annot=True, 
            cmap="seismic", vmin=-1, vmax=1)             # Color, min value -1, max value 1
plt.show()

- Create a boxplot of the columns: 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp' **1 point**

In [ ]:
# Since our variables have very different range, we do separate boxplots (different y-axis)
food_data.plot(
    kind='box', 
    subplots=True, 
    sharey=False,                                         # False to get different y-axis
    figsize=(12, 4),
    title = 'Box plots of yield, climate variables, and production factors'
)
plt.subplots_adjust(wspace=0.5)                           # Increase spacing between subplots
plt.show()

- Create a pairplot of the columns: 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp' **1 point**

In [ ]:
sns.pairplot(food_data) 
plt.suptitle('Pair plot of yield, climate variables, and production factors', y=1.02)
plt.show()

From the pair plot and correlation matrix, we notice that there are not obvious relations between the yields and our numerical features (pesticides, temperature, and rainfall). Our prediction exercise will thus be challenging... 

- Feel free to pursue your exploration to better understand your dataset. Although not graded, this might help you better understanding the problem and answer the following questions.

It seems that we have duplicate rows in our dataset:

In [ ]:
food[food.duplicated(keep=False)]

Let's explore a bit more. We'll use interactive features to facilitate our exploration:

In [ ]:
@interact
def interactive_extract(country = country_list, item = item_list, year = food.Year.unique()):
    return(food.loc[(food['Area']==country)& (food['Item']==item) & (food['Year']==year)])

We notice that for some countries we have several observations for the same year and item. However, not all of them are duplicates. For instance, if we select Brazil and any item and year, we notice that the temperature is not the same for all observations. We can assume that the duplicates are not actual errors but potentially refer to different locations. For now, we'll thus proceed by keeping all observations. However, if our goal was to design an actual predictive model, we would need to further explore the original data to better understand why we have various values of average temperature, and act accordingly...

### Question 2: Multivariate regression 

We will try to predict the crop yields (column 'hg/ha_yield') using as features: 'Item', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp'

- Extract your features and outcome **1 point**

In [ ]:
X = food[['Item', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp']]
y = food[['hg/ha_yield']]

- Split between training and test set **1 point**

*Note*: Use as option: `test_size=0.2`, `random_state=42`, `shuffle=True`

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

- Encode the column 'Item' using `LabelEncoder` **1 point**

*Note*: After training your encoder, you need to transform the values of both the training and test set

In [ ]:
# Extract the column of interest
item = X_train['Item'].values
item_test = X_test['Item'].values

# Define the encoder
le = LabelEncoder()

# Fit the encoder
le.fit(item)

# Transform the train and the test set
X_train = X_train.assign(Item=le.transform(item))
X_test = X_test.assign(Item=le.transform(item_test))

Let's check how the Item column was transformed:

In [ ]:
X_train.head(5)

- Rescale your features using `MinMaxScaler` **1 point**

In [ ]:
# Define the scaler
scaler = MinMaxScaler()

# Fit the scaler
scaler.fit(X_train)

# Transform the train and the test set
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

- Build and train a multivariate linear regression model **1 point**

In [ ]:
# Set up the model
model = LinearRegression()

# Train our model
model.fit(X_train, y_train)

- What is the $R^2$, mean absolute error, and mean square error on the training set? On the test set? What do you think? **1 point**

We first define a function to compute the various metrics, before using it and printing the result:

In [ ]:
def regression_metrics(model, X_train, X_test, y_train, y_test):
    """This function returns the MAE, MSE and R2 of a regression model, on the training and test set"""
    # Predict training and test set values
    predict_train = model.predict(X_train)
    predict_test = model.predict(X_test)
    # Metrics on training set
    mae_train = mean_absolute_error(y_train, predict_train)
    mse_train = mean_squared_error(y_train, predict_train)
    r2_train = r2_score(y_train, predict_train)
    # Metrics on test set
    mae_test = mean_absolute_error(y_test, predict_test)
    mse_test = mean_squared_error(y_test, predict_test)
    r2_test = r2_score(y_test, predict_test)
    # Return metrics
    return mae_train, mse_train, r2_train, mae_test, mse_test, r2_test 

In [ ]:
# Call our function
metrics_multi = regression_metrics(model, X_train, X_test, y_train, y_test)

# Print result
print(f"MAE test set: {metrics_multi[3]:0.1f}; MAE training set: {metrics_multi[0]:0.1f};")
print(f"MSE test set: {metrics_multi[4]:0.1f}; MSE training set: {metrics_multi[1]:0.1f};")
print(f"R\u00b2 test set: {metrics_multi[5]:0.3f}; R\u00b2 training set: {metrics_multi[2]:0.3f};" )

Our model leads to poor predictions. We can visualize our true vs predicted values:

In [ ]:
# Dataframe with true and predicted values
df_test_prediction = pd.DataFrame(y_test.values, columns = ['true'])
df_test_prediction['predicted'] = model.predict(X_test)

# Plot
plt.figure(figsize=(4,4))
sns.scatterplot(df_test_prediction, x='true', y='predicted')
plt.plot([0, 500000], [0, 500000],'k--')
plt.title('Predicted vs True yield value, multivariate regression')
plt.xlabel('True yield value')
plt.ylabel('Predicted yield value')
plt.show()

### Question 3: Polynomial features regression

We will try to improve the quality of our prediction using `PolynomialFeatures`.

- Write a function that is using as inputs the degree of polynomial features (an integer), the training and test set (for your features and outcome), and return the $R^2$, mean absolute error, and mean square error on the training and on the set of a polynomial feature regression **3 points**

*Hint:* You do not need to include in your function the splitting, encoding and scaling since we will reuse the ones set created before (as before). Your function should transform your training and test set to integrate polynomial features, then build and train your model, before calculating the various error metrics. 

In [ ]:
def model_poly(degree, X_train, X_test, y_train, y_test):
    """This function returns the MAE, MSE, and R2 of a polynomial feature regression, 
    on the training and test set, using as input the chosen degree, training, and test set"""
    # Initiate model with chosen number of degree
    poly = PolynomialFeatures(degree)
    # Transform our training and test set
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    # Build and train model
    model_poly = LinearRegression(fit_intercept=False)
    model_poly.fit(X_train_poly, y_train)
    # Use regression_metrics function to return metrics
    metrics_poly = regression_metrics(model_poly, X_train_poly, X_test_poly, y_train, y_test)
    return metrics_poly 

Let's check our function with 2 degrees:

In [ ]:
# Call our function
metrics_poly_2 = model_poly(2, X_train, X_test, y_train, y_test)

# Print result
print(f"MAE test set: {metrics_poly_2[3]:0.1f}; MAE training set: {metrics_poly_2[0]:0.1f};")
print(f"MSE test set: {metrics_poly_2[4]:0.1f}; MSE training set: {metrics_poly_2[1]:0.1f};")
print(f"R\u00b2 test set: {metrics_poly_2[5]:0.3f}; R\u00b2 training set: {metrics_poly_2[2]:0.3f};" )

- What are the the $R^2$, mean absolute error, and mean square error on the training and on the set of a polynomial features regression with degree = 3? With degree = 7? **1 point**

In [ ]:
# Call our function
metrics_poly_3 = model_poly(3, X_train, X_test, y_train, y_test)   # Polynomial features with 3 degrees

# Print result
print("Performance metrics for polynomial features with degree 3:")
print(f"MAE test set: {metrics_poly_3[3]:0.1f}; MAE training set: {metrics_poly_3[0]:0.1f}")
print(f"MSE test set: {metrics_poly_3[4]:0.1f}; MSE training set: {metrics_poly_3[1]:0.1f}")
print(f"R\u00b2 test set: {metrics_poly_3[5]:0.3f}; R\u00b2 training set: {metrics_poly_3[2]:0.3f}" )

In [ ]:
# Call our function
metrics_poly_5 = model_poly(5, X_train, X_test, y_train, y_test)   # Polynomial features with 5 degrees

# Print result
print("Performance metrics for polynomial features with degree 5:")
print(f"MAE test set: {metrics_poly_5[3]:0.1f}; MAE training set: {metrics_poly_5[0]:0.1f}")
print(f"MSE test set: {metrics_poly_5[4]:0.1f}; MSE training set: {metrics_poly_5[1]:0.1f}")
print(f"R\u00b2 test set: {metrics_poly_5[5]:0.3f}; R\u00b2 training set: {metrics_poly_5[2]:0.3f}" )

- Plot the evolution of the MSE on the training set for a polynomial feature regression model when the degree goes from 2 to 10. On the same figure, plot the MSE on the test set for a polynomial feature regression model, when the degree goes from 2 to 10. Which degree would you choose and why? **2 points**

In [ ]:
degree = range(2,11)
mse_poly_train = []
mse_poly_test = []

# We loop through our number of degrees, using our model_poly function, and storing the results in lists
for d in degree:
    metrics_poly = model_poly(d, X_train, X_test, y_train, y_test)
    mse_poly_train.append(metrics_poly[1])
    mse_poly_test.append(metrics_poly[4])

In [ ]:
# Plot
plt.figure()
plt.plot(degree, mse_poly_train, label = 'Training performance')
plt.plot(degree, mse_poly_test, 'k--', label = 'Test performance')
plt.title('Evolution of MSE in function of polynomial degree', y=1.05)
plt.grid(visible = True)
plt.legend()
plt.xlabel('Polynomial feature degree')
plt.ylabel('Mean Square Errors')
plt.show()

When we increase the number of degrees, the MSE on both training and test set is decreasing until degree 9. For a degree of 10, the MSE increases, indicating that we probably have overfitting issues. The gain in performance also "stagnates" for a degree of 6. 

To decided on the number of degrees, we should consider the complexity of our model. We have 4 original features: Item, Rainfall, Pesticides, and Temperature. For a polynomial features of degree d, we create $ {4+d}\choose {d}$$ = \frac{(4+d)!}{4! d!}$ features:

In [ ]:
n_poly_features = pd.DataFrame(index=['Number of polynomial features'])

for d in degree:
    n_poly_features[str(d)] = [comb(4+d,d)]
    
n_poly_features

Given the tradeoff performance-complexity, we will use 6 degrees in the following. Let's print all regression metrics for polynomial features of degree 6. Instead of directly using our function, we also create a training and test set, to be reused in future questions.

In [ ]:
# Initiate model with 6 degrees
poly = PolynomialFeatures(6)
# Transform our training and test set
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
# Build and train model
model_poly = LinearRegression(fit_intercept=False)
model_poly.fit(X_train_poly, y_train)
# Use regression_metrics function to return metrics
metrics_poly_6 = regression_metrics(model_poly, X_train_poly, X_test_poly, y_train, y_test)

# Print result
print("Performance metrics for polynomial features with degree 6:")
print(f"MAE test set: {metrics_poly_6[3]:0.1f}; MAE training set: {metrics_poly_6[0]:0.1f}")
print(f"MSE test set: {metrics_poly_6[4]:0.1f}; MSE training set: {metrics_poly_6[1]:0.1f}")
print(f"R\u00b2 test set: {metrics_poly_6[5]:0.3f}; R\u00b2 training set: {metrics_poly_6[2]:0.3f}" )

### Question 4: Ridge and cross-validation

- Build, train, and evaluate a polynomial features regression model, with Ridge regularization, and cross validation. For number of degree, select the one that you picked before. How does your new model compares to your previous one? **3 points**

In [ ]:
# Set up the model
model_ridge_cv = RidgeCV(cv=9, fit_intercept=False)

# Use fit
model_ridge_cv.fit(X_train_poly, y_train)

# Use regression_metrics function to return metrics
metrics_ridgecv = regression_metrics(model_ridge_cv, X_train_poly, X_test_poly, y_train, y_test)

# Results in a dataframe
model_performance = pd.DataFrame(index=['MAE','MSE','R2'])
model_performance['Polynomial features degree 6']=[metrics_poly_6[3], metrics_poly_6[4], metrics_poly_6[5]]
model_performance['Ridge CV degree 6']=[metrics_ridgecv[3], metrics_ridgecv[4], metrics_ridgecv[5]]
model_performance

The performance of our model decreases. We did not have overfitting issues before.

### Question 5: One-Hot encoding

We will check how the encoding influenced our results.

- Split your original dataset between training and test set (using the same parameters as in Question 2). This time, encode the column 'Item' using `OneHotEncoder`. Finally, rescale your features. **1 point**

In [ ]:
# Split between training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Extract the Item column
item = X_train[['Item']]
item_test = X_test[['Item']]

# Define the encoder
ohe = OneHotEncoder()

# Fit the encoder
ohe.fit(item)

# Add encoded data to Dataframe
X_train[ohe.categories_[0]]=ohe.transform(item).toarray()
X_test[ohe.categories_[0]]=ohe.transform(item_test).toarray()

Let's check the results, and verify that the encoding worked:

In [ ]:
display(X_train.head(5))
X_test.head(5)

Our encoded data matches with our items (at least for the first five observations...). Let's drop the item column and rescale our features:

In [ ]:
# Drop item column
X_train = X_train.drop('Item', axis=1)
X_test = X_test.drop('Item', axis=1)

# Rescale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

- Build, train, and evaluate a polynomial features regression model, with the same number of degrees as before, but this time with the one-hot encoded data. How does your model compares to the polynomial features regression model (Question 3)? **2 points**

We need to be careful. Since we did one-hot encoding instead of label encoding, we increased the number of features from 4 to 13. By doing a polynomial feature transformation, we can thus very quickly reach a very high number of features and run into memory issues:

In [ ]:
n_poly_features_ohe = pd.DataFrame(index=['Number of polynomial features'])

for d in degree:
    n_poly_features_ohe[str(d)] = [comb(13+d,d)]
    
n_poly_features_ohe

Therefore, instead of a degree 6, we will use a degree 3: 

In [ ]:
# Initiate model with 3 degrees
poly = PolynomialFeatures(3)
# Transform our training and test set
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
# Build and train model
model_poly_ohe = LinearRegression(fit_intercept=False)
model_poly_ohe.fit(X_train_poly, y_train)
# Use regression_metrics function to return metrics
metrics_poly_ohe_3 = regression_metrics(model_poly_ohe, X_train_poly, X_test_poly, y_train, y_test)

# Result in dataframe
model_performance['Polynomial features degree 3, label encoder']=[metrics_poly_3[3], metrics_poly_3[4], metrics_poly_3[5]]
model_performance['Polynomial features degree 3, one-hot encoder']=[metrics_poly_ohe_3[3], metrics_poly_ohe_3[4], metrics_poly_ohe_3[5]]
model_performance

One-hot encoding performs better. That being said, it is not surprising since it also is the model with the highest number of parameters (560) compared to the other ones. As a comparison, with label encoding, we could use a degree 8 and still have less parameters, while having lower MSE and MAE.

*Note:* You should not use $R^2$ to compare models with different number of features

Let's visualize our predicted vs true values for the polynomial feature model with one-hot encoding...

In [ ]:
# Dataframe with true and predicted values
df_test_prediction['predicted ohe'] = model_poly_ohe.predict(X_test_poly)

# Plot
plt.figure(figsize=(4,4))
sns.scatterplot(df_test_prediction, x='true', y='predicted ohe')
plt.plot([0, 500000], [0, 500000],'k--')
plt.title('Predicted vs True yield value, Polynomial features degree 3 with one-hot encoding')
plt.xlabel('True yield value')
plt.ylabel('Predicted yield value')
plt.show()

It looks not too bad, even though we are still struggling with very high yield values...

### Additional exploration

#### Comparing scaling techniques

Here is a function that returns the performance of different scalers, encoders, and polynomial feature degrees. In addition of the MAE, MSE, and $R^2$, we also return another metrics, namely the [Mean Absolute Percentage Error (MAPE)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html) - we'll use this metrics in the next section.

In [ ]:
def ML_algos(degree, scaler, encoder, X, y):
    """This function returns the MAE, MSE, R2 and MAPE on the training and test set using as input:
           - the polynomial feature degree
           - the chosen scaler
           - the chosen encoder
           - the features set X
           - the outcome y"""
    
    # Splitting dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
    
    # Encoding
    enc = encoder
    item = X_train[['Item']]
    item_test = X_test[['Item']]
    if str(enc) == 'LabelEncoder()':
        enc.fit(item.values.flatten())
        X_train = X_train.assign(Item=enc.transform(item.values.flatten()))
        X_test = X_test.assign(Item=enc.transform(item_test.values.flatten()))
    else:
        enc.fit(item)
        X_train[enc.categories_[0]]=enc.transform(item).toarray()
        X_test[enc.categories_[0]]=enc.transform(item_test).toarray()
        X_train = X_train.drop('Item', axis=1)
        X_test = X_test.drop('Item', axis=1)
        
    # Scaling
    scal = scaler
    X_train = scal.fit_transform(X_train)
    X_test = scal.transform(X_test)
    
    # Polynomial features
    poly = PolynomialFeatures(degree)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Build and train model
    model = LinearRegression(fit_intercept=False)
    model.fit(X_train_poly, y_train)
    
    # Metrics
    metrics = regression_metrics(model, X_train_poly, X_test_poly, y_train, y_test)
    mape_train = mean_absolute_percentage_error(y_train, model.predict(X_train_poly))
    mape_test = mean_absolute_percentage_error(y_test, model.predict(X_test_poly))
    
    # Format output
    metrics_df = pd.DataFrame([[metrics[0],metrics[3]],
                               [metrics[1],metrics[4]],
                               [metrics[2],metrics[5]],
                               [mape_train, mape_test]], 
                                index = ['MAE','MSE','R2', 'MAPE'], 
                                columns = ['Training', 'Test'])
    
    return metrics_df

We can test our function:

In [ ]:
ML_algos(3, StandardScaler(), LabelEncoder(), X, y)

Let's compare the performance of different models:

In [ ]:
model_performance = pd.DataFrame()

degrees = range(1,4)
scalers = [MinMaxScaler(), StandardScaler(), RobustScaler()]
encoders = [LabelEncoder(), OneHotEncoder()]

for d in degrees:
    for e in encoders:
        for s in scalers:
            test_perf = ML_algos(d, s, e, X, y)[['Test']]
            model_performance[f'Model: Degree = {d}, {s}, {e}']=test_perf
            
model_performance.transpose()

The model performance is not much influenced by the scaling techniques.

#### Log-transform yields and pesticides

We have seen in our EDA that the yields and pesticides were highly skewed. Let's redo our analysis, but this time we'll first log-transform the yield and pesticide columns:

In [ ]:
# Extract features and outcome
X_log = X.copy()
y_log = y.copy()

# Log-transform
X_log.loc[:,'pesticides_tonnes'] = np.log(X_log['pesticides_tonnes'])
y_log.loc[:,'hg/ha_yield'] = np.log(y_log['hg/ha_yield'])

display(X_log.head(5))
y_log.head(5)

Let's compare the performance of various models:

In [ ]:
model_log_performance = pd.DataFrame()

degrees = range(1,4)
encoders = [LabelEncoder(), OneHotEncoder()]

for d in degrees:
    for e in encoders:
        test_perf = ML_algos(d, MinMaxScaler(), e, X, y)[['Test']]
        model_log_performance[f'Original Model: Degree = {d}, MinMaxScaler(), {e}']=test_perf       
        test_perf_log = ML_algos(d, MinMaxScaler(), e, X_log, y_log)[['Test']]
        model_log_performance[f'Log Model: Degree = {d}, MinMaxScaler(), {e}']=test_perf_log
            
model_log_performance.transpose()

Bingo, the performance does increase! Note that we cannot rely on MSE and MAE to compare the performance of our models with and without log-transformation of our outcome since these metrics depend on the scale of the outcome variable. On the other hand, [MAPE](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-percentage-error) is scaled by the maximum true value, hence allowing more meaningful comparison.

## Classification

Access to safe drinking-water is essential to health, a basic human right and a component of effective policy for health protection. However, for a least 3 billion people, the quality of the water they depend on is unknown due to a lack of monitoring (see [SDG Goal 6](https://sdgs.un.org/goals/goal6) "Ensure availability and sustainable management of water and sanitation for all"). 

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Sustainable_Development_Goal_6.png/800px-Sustainable_Development_Goal_6.png' width="200">

We will use data from the [Water Quality](https://www.kaggle.com/datasets/mssmartypants/water-quality) dataset to try to predict whether the water is safe to drink depending on the concentration of various minerals and microorganisms. Check the webpage to read a description of the features and get a better understanding of our problem.

### Question 6: Load and Discover the dataset

- Load the data in a dataframe. The url link is provided below. Display the first 10 observations and the types of data.

In [ ]:
url_water = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/waterQuality1.csv'

water = pd.read_csv(url_water)
display(water.head(10))
water.dtypes

- Display summary statistics of your dataset and a heatmap of your correlation matrix **1 point**

In [ ]:
pd.set_option('display.max_columns', None) # Display all columns
display(water.describe())

In [ ]:
# Correlation matrix
water_corr = water.corr()

# Heatmap
plt.figure(figsize=(13, 8))
plt.title("Correlation matrix")
sns.heatmap(water_corr.round(decimals=2), annot=True, 
            cmap="seismic", vmin=-1, vmax=1)             # Color, min value -1, max value 1
plt.show()

- Create a pairplot including the columns "arsenic", "cadmium", "chromium", "copper", "bacteria", "viruses", "lead", "nitrates", "mercury"; and color by the class "is_safe" **1 points**

In [ ]:
water_col = ['arsenic', 'cadmium', 'chromium', 'copper', 'bacteria', 'viruses', 'lead', 'nitrates', 'mercury', 'is_safe']
water_data = water.loc[:,water_col]

sns.pairplot(water_data, 
             hue ='is_safe',
             palette = 'deep') 
plt.suptitle('Pair plot of water features colored by potability', y=1.02, fontsize=18)
plt.show()

- Feel free to pursue your exploration to better understand your dataset. Although not graded, this might help you better understanding the problem and answer the following questions.

In [ ]:
# YOUR CODE HERE


### Question 7: Preprocessing

We will try to predict the class "is_safe", using all the other features.

- Extract your features and outcome. How many observations do we have of class 0 and of class 1? **1 point**

In [ ]:
# Extract features and outcome
X = water.drop(columns='is_safe')
y = water['is_safe']

# Count observations of class 0 and 1
obs_0 = y.value_counts()[0]
obs_1 = y.value_counts()[1]
print(f'Our dataset contains {obs_0} observations of class 0 and {obs_1} observations of class 1.')

- Split between training and test set **1 point**

*Note*: Use as parameters for splitting: `test_size=0.2`, `random_state=39`, `shuffle=True`

In [ ]:
#Split data set into a train and a test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39, shuffle=True)

- Rescale your features using `StandardScaler` **1 point**

In [ ]:
# Define the scaler
scaler = StandardScaler()

# Fit the scaler
scaler.fit(X_train)

# Transform the train and the test set
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Question 8: Logistic Regression

- Build and train a logistic regression classifier, using as parameters `penalty='l2'`, `solver='lbfgs'`, `max_iter=1000` **1 point**

In [ ]:
# Set up our model
model_logi = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)

# Fit our model
model_logi.fit(X_train, y_train)

- Compute the accuracy on the training and test set. Compare it to the default rate. **1 point** 

In [ ]:
# Accuracy on the training set
acc_logi_train = model_logi.score(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set: {:.4f}'
     .format(acc_logi_train))

# Accuracy on test set
acc_logi_test = model_logi.score(X_test, y_test)
print('Accuracy of Logistic regression classifier on test set: {:.4f}'
     .format(acc_logi_test))

# Default rate
defaultrate = max(obs_0, obs_1)/(water["is_safe"].shape[0])
print(f'Default rate = {defaultrate:0.4f}')

We have unbalanced classes, class 0 representing 88.6% of observations, and thus the default rate is very high. Still, our accuracy is larger than the default rate. In addition, it seems that we do not have overfitting issues since the accuracy on the test set is similar (actually slightly larger) than the training set one.

- Plot a heatmap of the confusion matrix. Class 1 is the positive class. How many false positive did we obtain? **1 point**

In [ ]:
# Predict on test set
y_pred_logi = model_logi.predict(X_test)

# Confusion matrix
plt.figure(figsize=(3, 2.5))
sns.heatmap(confusion_matrix(y_test, y_pred_logi), annot=True, cmap='Blues', fmt='.4g')
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.title('Confusion Matrix');

We obtain 21 false positive (predicted class 1 while true label is 0). These values are problematic since we do not want to predict that the water is "safe" when it is not...

- What is the precision, recall, and f1 score of class 1? Interpret the result. **1 point**

In [ ]:
precision_logi = precision_score(y_test, y_pred_logi)   # Precision
recall_logi = recall_score(y_test, y_pred_logi)         # Recall
f1_logi = f1_score(y_test, y_pred_logi)                 # f1-score

print(f'The precision for class 1 (safe water) is: {precision_logi:0.3f}')
print(f'The recall for class 1 is: {recall_logi:0.3f}')
print(f'The F1 score for class 1 is: {f1_logi:0.3f}')

We already observed from the confusion matrix that our model was not performing well at predicting class 1, i.e., we have a low recall. On the other hand, the precision is larger. It is good news since our objective is to predict when the water is safe, i.e., we favor higher precision and lower recall. Still, our model needs improvements...

- Build and train a logistic regression classifier with cross-validation, using 5 folds **1 point**

In [ ]:
# Set up our model
model_logi_cv = LogisticRegressionCV(penalty='l2', solver='lbfgs', cv=5, max_iter=1000)

# Fit our model
model_logi_cv.fit(X_train, y_train)

- Plot a heatmap of the confusion matrix. Compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your model without cross-validation? **1 point**

*Note*: You can, but not have to, create a function to calculate your evaluation metrics since we will perform the same operation later on.

In [ ]:
# Confusion matrix
plt.figure(figsize=(3, 2.5))
sns.heatmap(confusion_matrix(y_test, model_logi_cv.predict(X_test)), annot=True, cmap='Blues', fmt='.4g')
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

Let's create a function to compute the accuracy on training set and test set, as well as precision, recall, and f1 score of class 1.

In [ ]:
def classification_metrics(model, X_train, X_test, y_train, y_test):
    """This function returns the accuracy on training set and test set, 
    as well as precision, recall, and f1 score of class 1,
    of a classifier"""
    # Accuracy on the training set
    train_accuracy = model.score(X_train, y_train)
    # Accuracy on test set
    test_accuracy = model.score(X_test, y_test) 
    # Prediction
    y_pred = model.predict(X_test)
    # Precision
    precision = precision_score(y_test, y_pred)
    # Recall
    recall = recall_score(y_test, y_pred)
    # f1
    f1 = f1_score(y_test, y_pred)
    # Return output
    return train_accuracy, test_accuracy, precision, recall, f1

In [ ]:
# Compute metrics model with cross validation
model_logi_cv_metrics = classification_metrics(model_logi_cv, X_train, X_test, y_train, y_test)

# Gather results in a dataframe:
model_logi_metrics = [acc_logi_train, acc_logi_test, precision_logi, recall_logi, f1_logi]
model_compar = pd.DataFrame(model_logi_metrics,
                    index = ['Train accuracy', 'Test accuracy', 'Precision', 'Recall', 'f1'], 
                    columns=['Logistic regression'])
model_compar['Logistic regression CV']=model_logi_cv_metrics
model_compar

We obtain exactly the same results as before, except for the accuracy on the training set. The reason is that, since we have no overfitting, our model performs best without regularization. Hence, in both case, the penalty parameter for the regularization is low, and our optimal solution is almost not influenced by the regularization. For instance, we can check the coefficients of the features in the decision function:

In [ ]:
coeff_models = pd.DataFrame(model_logi.coef_.flatten(), 
                            columns = ['Logistic regression'],
                            index = X.columns)
coeff_models['Logistic regression CV'] = model_logi_cv.coef_.flatten()
coeff_models

### Question 9: KNN classifier

- Build and train a KNN classifier with parameters `n_neighbors=7`, `p=2`, `weights='uniform'` **1 point**

In [ ]:
# Set up our model
model_kNN = KNeighborsClassifier(n_neighbors=7, p=2, weights='uniform')

# Fit our model
model_kNN.fit(X_train, y_train)

- Plot a heatmap of the confusion matrix. Compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your previous models? **1 point**

In [ ]:
# Confusion matrix
plt.figure(figsize=(3, 2.5))
sns.heatmap(confusion_matrix(y_test, model_kNN.predict(X_test)), annot=True, cmap='Blues', fmt='.4g')
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Metrics
model_kNN_metrics = classification_metrics(model_kNN, X_train, X_test, y_train, y_test)

# Add to dataframe
model_compar['KNN classifier']=model_kNN_metrics
model_compar

The accuracy, precision, recall, and f1 increased. We are on the right track! That being said, the number of false positive did not change much (19 vs 21 before). The improvement is mostly due to better prediction of class 1.

- Use `GridSearchCV` to explore different parameters for your model: `n_neighbors` between 1 and 11, `p` between 1 and 3, and `weights` either 'uniform' or 'distance' **1 point**

In [ ]:
# Define parameters to test
grid = {'n_neighbors':np.arange(1,12),     # array from 1 to 11 neighbors
        'p':np.arange(1,3),                # array from 1 to 3, distance metrics
        'weights':['uniform','distance']   # weights
       }

# Define and fit model
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, grid, cv=7)
knn_cv.fit(X_train, y_train)

# Print results
print("Hyperparameters:", knn_cv.best_params_)
print("Best model: ", knn_cv.best_estimator_)

- For your "optimal" model, compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your previous models? **1 point** 

In [ ]:
# Metrics
model_kNN_cv_metrics = classification_metrics(knn_cv.best_estimator_, X_train, X_test, y_train, y_test)

# Add to dataframe
model_compar['KNN with Grid Search']=model_kNN_cv_metrics
model_compar

Interestingly, the precision increased while the recall stayed constant. We can check the confusion matrix to confirm that the number of false positive decreased:

In [ ]:
# Confusion matrix
plt.figure(figsize=(3, 2.5))
sns.heatmap(confusion_matrix(y_test, knn_cv.predict(X_test)), annot=True, cmap='Blues', fmt='.4g')
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

### Question 10: Decision Trees

- Build and train a Decision Tree with parameters `criterion = 'gini'`, `max_depth = 3` **1 point**

In [ ]:
# Create model 
model_tree = DecisionTreeClassifier(criterion = 'gini', max_depth = 3)

# Fit model
model_tree.fit(X_train, y_train)

- Plot a heatmap of the confusion matrix. Compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your previous models? **1 point**

In [ ]:
# Confusion matrix
plt.figure(figsize=(3, 2.5))
sns.heatmap(confusion_matrix(y_test, model_tree.predict(X_test)), annot=True, cmap='Blues', fmt='.4g')
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Metrics
model_tree_metrics = classification_metrics(model_tree, X_train, X_test, y_train, y_test)

# Add to dataframe
model_compar['Decision Tree']=model_tree_metrics
model_compar

It keeps getting better! Our accuracy on the test set is quite good, 94%, and we only have 9 false positive, resulting in a precision of 91%. The recall is now also above 50%, i.e., we have more true positive than false negative. Note that we shouldn't be surprised by these good results. Indeed, for the water to be safe, it requires pollutants and microorganisms to be below a certain level. It's exactly a task for a decision classifier!

- Visualize your Decision Tree **1 point**

In [ ]:
# Decision tree plot
plt.figure(figsize=(20,7))
plot_tree(model_tree, feature_names= X.columns, filled=True, fontsize=12)
plt.title('Decision Tree visualization', fontsize=15)
plt.show()

Our tree is using the aluminium content as first split, and then proceed with aluminium and cadmium at the second level, and so on. One leaf has more observations belonging to class 1 (in blue). It is obtained for low values of aluminium, cadmium, and perchlorate. 

Note: the negative values are because we scaled our features using StandardScaler().

- Use `GridSearchCV` to explore different parameters for your model: `criterion` either 'gini' or 'entropy' and `max_depth` between 1 and 7 **1 point**

In [ ]:
# Define parameters to test
grid_tree = {'criterion':['gini','entropy'] ,     # criterion
        'max_depth':np.arange(1,8),               # array from 1 to 7, maximum depth
       }

# Define and fit model
dec_tree = DecisionTreeClassifier()
dec_tree_cv = GridSearchCV(dec_tree, grid_tree, cv=5)
dec_tree_cv.fit(X_train, y_train)

# Print results
print("Hyperparameters:", dec_tree_cv.best_params_)
print("Best model:", dec_tree_cv.best_estimator_)

- For your "optimal" model, compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your previous models? **1 point** 

In [ ]:
# Metrics
model_tree_cv_metrics = classification_metrics(dec_tree_cv.best_estimator_, X_train, X_test, y_train, y_test)

# Add to dataframe
model_compar['Tree with Grid Search']=model_tree_cv_metrics
model_compar

The model obtained with Grid Search has a depth of 7 and is using "entropy" instead of "gini". Hence, the accuracy on test set increases to 96.5%. The recall jumps to 75% while the precision stays more or less the same, which means our model got better at predicting class 1 (less false negative), but not at avoiding false positive:

In [ ]:
# Confusion matrix
plt.figure(figsize=(3, 2.5))
sns.heatmap(confusion_matrix(y_test, dec_tree_cv.predict(X_test)), annot=True, cmap='Blues', fmt='.4g')
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()